<a href="https://colab.research.google.com/github/Astridgiraldo/RH/blob/main/Limpieza_y_transformacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Una empresa que tiene alrededor de 4000 empleados tiene una tasa de retiros de alrededor del 15% anual. Esto quiere decir que el 15% de los empleados en un año se retiran de la compañía por diferentes motivos. Los directivos de la empresa consideran que este nivel de retiros es muy alto, ya que trae altos costos para la empresa como: el costo de contratación que incluye todos los procesos desde el reclutamiento, la selección, la contratación y adicionalmente existe un costo relacionado con la capacitación, la curva de aprendizaje y las contrataciones no efectivas que son personas que se retiran de la empresa en el periodo de prueba. Adicionalmente, el retiro de las personas conlleva otras implicaciones negativas para la empresa que se presentan a continuación:

Los proyectos en los que los empleados trabajan se atrasan y se pueden ver comprometidas las fechas establecidas y esto puede llegar a afectar satisfacción de clientes y usuarios.
El trabajo del área de selección se aumenta y por lo tanto implica tener un área de mayor tamaño para el reclutamiento de las personas que se van.
Los empleados que permanecen tienen que trabajar más para cubrir las labores de las personas que se retiran y para la capacitación de las nuevas que llegan.
El conocimiento que tenían las personas y su experiencia se pierde y, debido a esto, el tiempo para ejecutar procesos aumenta.